Pytorch implementation of our problem.

The following problem will be solved in each node of a dynamic programming problem:

find n Actions such that they constitute a local nash equilibrium

input = Action
loss = stochatic salvo combat model: probabilty of winning

restart the process after updating the state with each of these Actions.

In [1]:
import torch
from torch.autograd import grad, Function
import numpy as np
import time

In [ ]:
# # global variables
# N_Technologies = 3
# N_Capabilities = 6
# Horizon = 5

# # Used in TRL calculations
# I = 25
# D = 0
# CAPABILITYMATRIX = torch.rand(N_Technologies,N_Capabilities,2) # assuming differnt conversion for each of the players, informed by specific scenario 



In [ ]:
# #helper functions 

# def Update_State(State,Action):
    
#     #UpdateValue = randomness(Action) #implement randomness
#     UpdateValue = Action
    
#     return torch.add(State,UpdateValue)

# def TechnologyReadiness(State):
#     global D,I
#     TRL = torch.pow(1+torch.exp(-State*(1/I)+D),-1)
#     return TRL

# def TechToCapa(State):
#     #Capabilities = np.matmul(np.transpose(State),CAPABILITYMATRIX)
#     TechnologyReadinessLevel = TechnologyReadiness(State)
#     print(TechnologyReadinessLevel)

    
#     Capabilities = torch.empty((N_Capabilities,2))

#     for i in range(2):
#         Capabilities[:,i] = torch.transpose(TechnologyReadinessLevel[:,i],0,-1) @ CAPABILITYMATRIX[:,:,i]
        
#     return Capabilities
 


In [ ]:
# #should give probabilities that [player 1, player 2] wins the battle. 
# def torchBattle(capabilities):
#     results = torch.div(torch.sum(capabilities,dim=0) , torch.sum(capabilities))
#     return results
    

In [143]:
class TorchGame():
    def __init__(self, N_Technologies =3, N_Capabilities = 6, Horizon = 5, N_actions = 5, N_actions_startpoint = 100, I=25, D = 0) -> None:
        torch.manual_seed(1337)
        # global variables
        self.N_Technologies = N_Technologies
        self.N_Capabilities = N_Capabilities
        self.Horizon = Horizon
        self.N_actions_startpoint = N_actions_startpoint
        self.N_actions = N_actions
        
        # Used in TRL calculations
        self.I = I
        self.D = D
        
        #
        self.CAPABILITYMATRIX = torch.rand(N_Technologies,N_Capabilities,2) # assuming differnt conversion for each of the players, informed by specific scenario 

        
        #creating the initalState
        st = torch.rand(N_Technologies,2)
        divisor = 0.01*torch.sum(st,0) # sum to 100
        self.InitialState = torch.divide(st,divisor)
        
        self.History = []
        self.Q = []
    
    def Update_State(self,State,Action):
        #UpdateValue = randomness(Action) #implement stochasticity
        UpdateValue = Action
        
        newState = torch.add(State,UpdateValue)
        newState.requires_grad_(True)
        
        return newState

    def TechnologyReadiness(self,State):
        
        TRL = torch.pow(1+torch.exp(-State*(1/self.I)+self.D),-1)
        TRL.requires_grad_(True)
        return TRL

    def TechToCapa(self,State):
        
        TechnologyReadinessLevel = self.TechnologyReadiness(State)
        
        
        Capabilities = torch.empty((self.N_Capabilities,2))

        for i in range(2):
            Capabilities[:,i] = torch.transpose(TechnologyReadinessLevel[:,i],0,-1) @ self.CAPABILITYMATRIX[:,:,i]
        Capabilities.requires_grad_(True)

            
            
        return Capabilities
    
    def Battle(self,Capabilities):
        results = torch.div(torch.sum(Capabilities,dim=0) , torch.sum(Capabilities))
        return results
    
    def OptimizeAction(self, State,Action): #this should use the battle function

        #this is really the only place where the whole pytorch thing is required. The rest can be base python or numpy
        eps = 1E-2
        iteration = 0
        
        learningRate = 1
        gradFlipper = torch.transpose(torch.tensor([ [1]*self.N_Technologies , [-1] * self.N_Technologies]),0,-1)

        act_n = Action.clone().detach().requires_grad_(True)
        dA = 1
        while torch.norm(dA) > eps and iteration < 50:
            
            trl = torch.pow(1+torch.exp(-torch.add(State,act_n)*(1/self.I)+self.D),-1)
            
            
            trl_temp = torch.unsqueeze(torch.transpose(trl,0,-1),1)
            capa_temp = torch.transpose(torch.transpose(self.CAPABILITYMATRIX,2,0),1,2)
        

            
            capabilities = torch.matmul(trl_temp,capa_temp )
            score = torch.div(torch.sum(capabilities,dim=0) , torch.sum(capabilities))
            
            score.backward(torch.ones_like(score))
            
            # print(gradAct.is_leaf)
            
            dA = act_n.grad
            act_n = torch.add(act_n + dA * gradFlipper * learningRate)
            
            print(f"norm(dA) = {torch.norm(dA)}, P1 winprob = {score}")
            
            iteration +=1 

    
        
        return (Action + torch.rand_like(Action)) * .5
        
    def FilterActions(self, Actions): #keep optimization trajectories that converged, and filter out "duplicates" s.t., tol < eps
        return Actions[:self.N_actions]

    def GetActions(self,State):
        
        ActionStartPoints = torch.rand(self.N_Technologies,2,self.N_actions_startpoint)
        
        NashEquilibria = []
        for i in range(self.N_actions_startpoint):
            init_action = ActionStartPoints[:,:,i]#.clone().detach().requires_grad_(True)
            NE_action = self.OptimizeAction(State,  init_action)
            NashEquilibria.append(NE_action)
            
         
        return self.FilterActions(NashEquilibria)
    
    def Main(self):
        start = time.time()
        self.Q.append((self.InitialState,0))
        
        while (len(self.Q) > 0 and time.time() - start < 10):
            st,t = self.Q.pop() #the state which we are currently examining
            #print(t)
            act = self.GetActions(st) # small number of nash equilibria
            for a in act:
                self.History.append((st,a)) # adding the entering state and the exiting action to history, reward should probably also be added. 
                                          
                
                st_new = self.Update_State(st,a) #the resulting states of traversing along the nash equilibrium
                if t+1 < self.Horizon:
                    self.Q.append((st_new,t+1))
                    
        return self.History
                
             

            
           
FullGame = TorchGame(N_Technologies=21,Horizon=4,N_actions=3)
hist = FullGame.Main()
#print(len(hist))

AttributeError: 'int' object has no attribute 'dim'

In [ ]:
tensor1 = torch.randn(2, 1, 21)
tensor2 = torch.randn(2, 21, 6)
torch.matmul(tensor1, tensor2).size()

In [119]:
a = 5
b = 10
#c = 12

A  = torch.rand(b,a)
x = torch.rand(b,1).requires_grad_()
z = torch.rand(a,1)

y = torch.exp(torch.transpose(A,0,-1) @ x) * z

y.backward(torch.ones_like(y))

print(torch.sum(A,1))
print(x.grad)


tensor([3.2033, 3.6669, 1.9358, 2.8554, 1.4877, 3.2222, 1.8596, 2.2637, 2.8329,
        2.5007])
tensor([[44.0411],
        [51.8836],
        [27.3188],
        [43.0039],
        [24.6529],
        [51.8322],
        [26.3758],
        [30.0112],
        [40.4161],
        [39.1854]])


In [134]:
CAPABILITYMATRIX

tensor([[[0.1237, 0.6139],
         [0.8393, 0.1400],
         [0.4973, 0.1789],
         [0.5344, 0.3982],
         [0.5101, 0.1695],
         [0.3680, 0.7682]],

        [[0.1063, 0.7549],
         [0.1615, 0.1600],
         [0.7457, 0.5889],
         [0.1432, 0.8644],
         [0.6009, 0.1288],
         [0.3015, 0.7617]],

        [[0.7241, 0.2301],
         [0.2023, 0.6491],
         [0.7809, 0.7176],
         [0.6532, 0.9485],
         [0.8844, 0.3448],
         [0.3793, 0.4971]],

        [[0.7479, 0.9166],
         [0.5431, 0.7336],
         [0.1709, 0.5611],
         [0.8779, 0.1661],
         [0.6270, 0.8584],
         [0.3914, 0.0303]],

        [[0.6999, 0.1528],
         [0.8424, 0.7754],
         [0.0467, 0.5553],
         [0.4903, 0.9525],
         [0.8764, 0.2972],
         [0.0605, 0.6580]],

        [[0.8613, 0.2906],
         [0.2896, 0.3111],
         [0.5234, 0.8249],
         [0.5707, 0.9190],
         [0.8812, 0.2342],
         [0.0894, 0.3608]],

        [[0.8135

In [151]:
N_Technologies = 21
N_Capabilities = 6
I = 25
D = 0

CAPABILITYMATRIX = torch.rand(N_Technologies,N_Capabilities,2)
State = torch.rand(N_Technologies,2)
Action = torch.rand(N_Technologies,2)


Stat = State#torch.tensor(State)

     
gradAct = Action.clone().detach().requires_grad_(True)

Stat = torch.add(Stat,gradAct)
trl = torch.pow(1+torch.exp(-Stat*(1/I)+D),-1)

print(trl.grad_fn)
trl_temp = torch.unsqueeze(torch.transpose(trl,0,-1),1)
capa_temp = torch.transpose(torch.transpose(CAPABILITYMATRIX,2,0),1,2)



capabilities = torch.squeeze(torch.matmul(trl_temp,capa_temp ))
score = torch.sum(capabilities,dim=1) / torch.sum(capabilities)



score.backward(torch.ones_like(score))

# print(gradAct.is_leaf)

dA = gradAct.grad        
print(dA)
print(score)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([0.5368, 0.4632], grad_fn=<DivBackward0>)


In [149]:
capabilities.size()

torch.Size([2, 6])

In [137]:
            gradFlipper = torch.transpose(torch.tensor([ [1]*N_Technologies , [-1] * N_Technologies]),0,-1)
            gradFlipper.size()

torch.Size([21, 2])

In [ ]:
# #simple test Case 

# State_0 = torch.rand(N_Technologies,2)
# divisor = 0.01*torch.sum(State_0,0) # sum to 100
# State_0 = torch.divide(State_0,divisor)


# Action_0 = torch.rand(N_Technologies,2)
# divisor = 0.2*torch.sum(Action_0,0) # sum to 5
# Action_0 = torch.divide(Action_0, divisor)

# print(State_0)

# State_1 = Update_State(State_0,Action_0)
# print(State_1)

# Capabilities_1 = TechToCapa(State_1)
# print(Capabilities_1)

# results = torchBattle(Capabilities_1)

# print(results)

In [ ]:
# 
# #import matplotlib.pyplot as plt 

# nRange = range(4,200,20)
# results = np.zeros((len(nRange),2))

# for i,n in enumerate(nRange):
#     bigMatrix = torch.rand(n,n).cuda()

#     start = time.time()

#     torch.linalg.eig(bigMatrix)
#     end = time.time()

#     gpuTime = end-start

#     start = time.time()
#     for _ in range(50):

#         torch.linalg.eig(bigMatrix)
#     end = time.time()
#     cpuTime = end-start
    
#     results[i,:] = [gpuTime,cpuTime]

# print(results)



In [ ]:
# start = time.time()
# time.sleep(1)
# end = time.time()
# total = end-start
# total

In [32]:
# random points in hypersphere

def randomPointsInShere(nPoints,nDim,rad):
    #https://en.wikipedia.org/wiki/N-sphere#Spherical_coordinates
    
    params = torch.rand(size = (nPoints,nDim+1))
    
    r = params[:,-1] * rad
    X = (r * torch.eye(nPoints)) @ torch.ones(nPoints,nDim) 

    for i in range(nDim-1):
        X[:,i] *= torch.cos(X[:,i])
        #print(f"c{i}")
        for j in range(i):
            X[:,i] *= torch.sin(X[:,j])
            #print(f"s{j}")
    
    for j in range(nDim):
            X[:,nDim-1] *= torch.sin(X[:,j])
            #print(f"s{j}")
    
    return X 


    
nPoints = 5
nDim = 2

X = randomPointsInShere(nPoints,nDim, 1)
print(X)

torch.norm(X,p=2,dim=1)

tensor([[3.6634e-01, 2.0182e-02],
        [5.2397e-01, 1.0925e-01],
        [4.5572e-01, 5.3390e-02],
        [1.5031e-01, 5.1849e-04],
        [1.2115e-01, 2.1760e-04]])


tensor([0.3669, 0.5352, 0.4588, 0.1503, 0.1212])

In [9]:
import pandas as pd
df_stat = pd.read_excel("config_files/State_Conversion.xlsx", sheet_name="StartingState",header=0, index_col=0)
print(df_stat)

df_capaMat = pd.read_excel("config_files/State_Conversion.xlsx", sheet_name="ConversionMatrix",header=0, index_col=0)
print(df_capaMat)

             PlayerA_y  PlayerB_y
sen_tec              4          4
col_sys_alg          1          1
tec_mob              5          5
cont_alg             3          3
loc_map              2          2
sen_fus              3          3
ai_ml                5          5
edg_com              3          3
com_net              5          5
ene_mgm              2          2
sim_mod              1          1
          sen_tec  col_sys_alg  tec_mob  cont_alg  loc_map  sen_fus  ai_ml  \
A,B             0            0        0         0        0        0      0   
Phi, Psi        9            3        3         1        9        9      3   
n_a,n_b         0            3        1         1        1        0      3   
p_a,p_b         3            9        9         9        3        1      1   
n_y,n_z         9            1        3         3        1        3      3   
p_y,p_z         9            9        9         3        3        9      9   
u,v             0            0        0       